# **Data Cleaning Notebook**

## Objectives

*   Evaluate missing data
*   Clean data

## Inputs

* outputs/datasets/collection/airplane_performance_study.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned

## Conclusions

 
  * Data Cleaning Pipeline
  * Drop Variables:  `['Model', 'Company', 'THR', 'SHP']`



---


# Change working directory

We need to change the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Collected data

In [ ]:
import pandas as pd
df_raw_path = "/workspace/data-driven-design/outputs/datasets/collection/airplane_performance_study.csv"
df = pd.read_csv(df_raw_path)
df.head(3)

# Data Cleaning - Part 1

## Assessing Missing Data Levels

This dataset is already cleaned thus we should be able to expect no missing data however `['THR', 'SHP']` do have apparent missing data in its columns. The reason for this is that "size" of an engine is measured with different quantities/units depending on ` Engine Type`. The jet is measured in quantity of force (unit: THR) and the piston and propjet (both propeller) is measured using power (unit: SHP). For this reason these units are split into two separate columns and a jet data point will then have N/A in the SHP column and a prop or propjet (turbo prop) will have a N/A in the THR column. 

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=['PercentageOfDataset'], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data
EvaluateMissingData(df)

## Features to drop

We will drop the following features:
* `'THR'` and `'THR'` since these are different quantities with different untis and therefore difficult to compare with each other.
* `'Model'` and `'Company'` since this meta data only serve as identifier variables


### Drop Variables

We drop values in this stage already since subsequents code otherwise would fail. 

List here the data cleaning approaches you want initially to try.
* Drop - `['Model', 'Company', 'THR', 'SHP']`

In [ ]:
# Columns to drop
columns_to_drop = ['Model', 'Company', 'THR', 'SHP']

# Dropping the columns
df_dropped = df.drop(columns=columns_to_drop)

# Overwriting to the droped data frame
df =  df_dropped

# Display the modified DataFrame
df.head(3)

# Data Exploration

## Correlation and PPS Analysis

In [ ]:
pip install ppscore

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps


def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)


Calculate Correlations and Power Predictive Score

In below cell I have to drop values to make the correlation below to work.

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df_dropped)

Display at Heatmaps

In [ ]:
df = pd.DataFrame(df_dropped)
print(df.dtypes)


Solve issue with "AttributeError: model 'numpy' has no attribute 'bool'. 'np.bool' was a deprecated alias for the builtin 'bool'..."

In [32]:
import numpy as np
np.bool = np.bool_

Solve issue that the plots were not being displayed

In [33]:
%matplotlib inline

We have added the Predictive Power Score (PPS) since PPS also can detect non-linear relationships that the traditional metrics like Pearson or Spearman correlations might miss.

Observations and conclusion from below heatmaps:

* We naturally see high correlation between features of the same kind, e.g. we see high correlation (0,85 - 0,88) between the "weight"-features like "All Up Weight" (AUW), Fuel Weight(FW) and "Manufacturer Empty Weight" (MEW) since both are "weight"-features. We also see high correlation between features in related kinds, e.g. the size of the airplane (Height, Length and Wing Span) are obvioously strongly related to airplanes weight (AUW, FW and MEW).

* Albeit not really interresting from an Airplane Design perspective these relationships can still be used to predict values. Note tha the relatinship between size and weight (due to structural and aerodynamic scale factors) might not be linear. 

* Interesting correlations are between: 0.8 - 1.0:



Running heatmaps with low threshold to allowe both **weakly**, **moderate** and **strongly** correlated features

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.3, PPS_Threshold =0.2,
                  figsize=(12,10), font_annot=10)

Now we run the correlation again with elevated thresholds as to only filter out both weakly and moderate correlation values leaving only **strongly** correlated features

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.7, PPS_Threshold =0.5,
                  figsize=(12,10), font_annot=10)

# Data Cleaning - Part 2

## Assessing Missing Data Levels

### Data Cleaning Summary

The following data cleaning actions have been taken:
* Drop - `['Model', 'Company', 'THR', 'SHP']`

### Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['Multi_Engine'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

# Push cleaned data to Repo

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

## Train Set

In [39]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

## Test Set

In [40]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)